In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns

from scipy.stats import poisson
import matplotlib.pyplot as plt

a)

State space $S \in (0,1,2,3,4)$, representing the amount of items on hand

Action space $A \in (0,1,2,3,4)$, representing the amount of items to be ordered at the end of the day

Reward function: $\begin{equation}
  r(X_n, X_{n+1}, A_n)=\begin{cases}
    100[X_n-X_{n+1}]^+ - 50a -100, & \text{if $A_n > 0$}.\\
    100[X_n-X_{n+1}]^+, & \text{if $A_n = 0$}.
  \end{cases}
\end{equation}$

representing the reward associated with the transition from state $X_n$ to state $X_{n+1}$ given action state $a$. If no delivery, i.e. $A_n = 0$, then no delivery cost or item cost associated with reward

Choice of action states $a$ are only permitted to the extent that the current state space $s$ contains enough inventory to allow for such selection

In [2]:
#5 states and actions

states, actions = [i for i in range(5)], [i for i in range(5)]

In [3]:
#poisson distribution of demand

B=np.zeros((5,5))
#i is the initial number of items
for i in range(5):
    #j is the new number after decay
    for j in range(1,i+1):
        #i-j decayed out of i, with lambda=4
        prob=poisson.pmf(i-j,2)
        B[j][i]=prob
    B[0][i]=1-np.sum(B[:,i]) 
p = B.T
p

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.86466472,  0.13533528,  0.        ,  0.        ,  0.        ],
       [ 0.59399415,  0.27067057,  0.13533528,  0.        ,  0.        ],
       [ 0.32332358,  0.27067057,  0.27067057,  0.13533528,  0.        ],
       [ 0.14287654,  0.18044704,  0.27067057,  0.27067057,  0.13533528]])

In [286]:
#reward function matrix

r = [np.zeros((5,5)) for i in range(5)]
def reward(x,y,a):
    if a > 0:
        return 100*max(0,x-y) - 50*a - 100
    else:
        return 100*(max(0,x-y))

for i in range(5):
    for j in range(5):
        for k in range(5):
            if i > j:
                r[i][j][k] = 0
            else:
                r[i][j][k] = reward(j,k,i)
r

[array([[   0.,    0.,    0.,    0.,    0.],
        [ 100.,    0.,    0.,    0.,    0.],
        [ 200.,  100.,    0.,    0.,    0.],
        [ 300.,  200.,  100.,    0.,    0.],
        [ 400.,  300.,  200.,  100.,    0.]]),
 array([[   0.,    0.,    0.,    0.,    0.],
        [ -50., -150., -150., -150., -150.],
        [  50.,  -50., -150., -150., -150.],
        [ 150.,   50.,  -50., -150., -150.],
        [ 250.,  150.,   50.,  -50., -150.]]),
 array([[   0.,    0.,    0.,    0.,    0.],
        [   0.,    0.,    0.,    0.,    0.],
        [   0., -100., -200., -200., -200.],
        [ 100.,    0., -100., -200., -200.],
        [ 200.,  100.,    0., -100., -200.]]),
 array([[   0.,    0.,    0.,    0.,    0.],
        [   0.,    0.,    0.,    0.,    0.],
        [   0.,    0.,    0.,    0.,    0.],
        [  50.,  -50., -150., -250., -250.],
        [ 150.,   50.,  -50., -150., -250.]]),
 array([[   0.,    0.,    0.,    0.,    0.],
        [   0.,    0.,    0.,    0.,    0.],
  

In [5]:
#need to create a subset of actions that you can take if you're in a given state
#if you're in state 4, you can't buy any items (must take action 0)

a_map = {i:[j for j in range(5-i)] for i in states}
a_map

{0: [0, 1, 2, 3, 4], 1: [0, 1, 2, 3], 2: [0, 1, 2], 3: [0, 1], 4: [0]}

In [6]:
#test for state 4 > action 0 > reward computation

test1 = np.array([ 0.14287654,  0.18044704,  0.27067057,  0.27067057,  0.13533528])
test2 = np.array([ 400.,  300.,  200.,  100.,    0.])
test2.dot(test1)

192.48589899999999

In [7]:
## test for state 0 > action 4 > reward computation

test1 = np.array([ 0.14287654,  0.18044704,  0.27067057,  0.27067057,  0.13533528])
test2 = np.array([ 100.,    0., -100., -200., -300.])
test1.dot(test2)

-107.51410099999998

In [8]:
#create reward vectors as conditional expectation of each action / state using
#the map of possible states resulting from given actions

#i.e. you were in state 3: you took action 0: expectation of reward = 178
#you were in state 4: you took action 0: expectation of reward = 192

#YOU START IN STATE 0, TAKE ACTION 4, END IN STATE 4 (TAKE EXPECTED VALUE HERE)

#new
exp_r = {}
for i, j in a_map.items():
    exp_r[i] = [r[k][i+k].dot(p[i+k]) for k in j]
exp_r

{0: [0.0,
  -63.533528323661272,
  -54.134113294645083,
  -71.801754912951424,
  -107.51410096280614],
 1: [86.466471676338728,
  -4.1341132946450792,
  -21.801754912951431,
  -57.51410096280614],
 2: [145.86588670535491, 28.198245087048569, -7.5141009628061326],
 3: [178.19824508704858, 42.485899037193867],
 4: [192.48589903719386]}

In [9]:
def value_iteration(iters, discount, verbose):
    
    policy = [np.zeros(5) for i in range(iters)]
    v = np.array([i[0] for i in exp_r.values()])

    if verbose == True:
        print('############\ninitial values:\n',v,'\n############')
        
    for i in range(iters):
        v_temp = np.zeros(5)
        for state in states:
            rewards = {}
            if verbose == True:
                print('\ncurrent state:', state, '\ncurrent value:', v[state])
            if verbose == True:
                print('available actions:', a_map[state],'\n')
            
            total = 0
            for action in a_map[state]:
                rewards[action] = exp_r[state][action] + (discount * v.dot(p[state + action]))

            if verbose == True:
                print('Actions/Rewards:\n', rewards)
            
            reward_max = max(rewards.values())
            v_temp[state] = reward_max

            best_action = [x==reward_max for x in rewards.values()].index(True)
            policy[i][state] = best_action
            
            if verbose == True:
                print('### Optimal Action: ', best_action,'###\n')

        v = v_temp

        
        if verbose == True:
            print('###############\nvalue update:', v, '\n###############')

    return v, policy[i]

In [11]:
value, policy = value_iteration(25,.99, False)
print('Value: ',value,'\n')
print('Policy: ',policy,'\n')

Value:  [  867.37996584   922.00895931  1007.50141453  1088.19238994  1167.37996584] 

Policy:  [ 4.  0.  0.  0.  0.] 



In [12]:
value, policy = value_iteration(100,.99, False)
print('Value: ',value,'\n')
print('Policy: ',policy,'\n')

Value:  [ 2481.18946473  2535.8184582   2621.31091342  2702.00188883  2781.18946473] 

Policy:  [ 4.  0.  0.  0.  0.] 



In [13]:
value, policy = value_iteration(400,.99, False)
print('Value: ',value,'\n')
print('Policy: ',policy,'\n')

Value:  [ 3845.32903897  3899.95803244  3985.45048766  4066.14146306  4145.32903897] 

Policy:  [ 4.  0.  0.  0.  0.] 



In [14]:
value, policy = value_iteration(1000,.99, False)
print('Value: ',value,'\n')
print('Policy: ',policy,'\n')

Value:  [ 3915.50843168  3970.13742515  4055.62988037  4136.32085577  4215.50843168] 

Policy:  [ 4.  0.  0.  0.  0.] 



In [15]:
value, policy = value_iteration(10000,.99, False)
print('Value: ',value,'\n')
print('Policy: ',policy,'\n')

Value:  [ 3915.67762067  3970.30661415  4055.79906936  4136.49004477  4215.67762067] 

Policy:  [ 4.  0.  0.  0.  0.] 



b)

This originally didn't work for me in python, so I used the below commands/matrices in matlab and got these results:


linprog(c',-A,-b)

Optimal solution found.

ans = 

1.0e+03 *

3.9157
3.9703
4.0558
4.1365
4.2157


This is consistent with the results from my value iteration in part a, but the linear program actually took significantly less iterations (8 vs over 1000) to converge


However, I tried using a method other than simplex (not sure if this was ok but I really didn't have much of a choice). I ended up using an interior point method optimization in python and that seemed to work better.

All of the code is below

In [160]:
v = np.array([i[0] for i in exp_r.values()])

In [639]:
c = np.array([1 for i in range(5)])
c

array([1, 1, 1, 1, 1])

In [646]:
b = exp_r[0] + exp_r[1] + [-102.2488] + exp_r[2] \
+ [-52.2488,-102.2488] + exp_r[3] + [-2.2488,-52.2488,-101.0458,192.4859] \
+ [47.7512, -2.2488, -51.0458, -100.3584]
b = [round(i,4) for i in b]
b = np.matrix(b)
b.T

matrix([[   0.    ],
        [ -63.5335],
        [ -54.1341],
        [ -71.8018],
        [-107.5141],
        [  86.4665],
        [  -4.1341],
        [ -21.8018],
        [ -57.5141],
        [-102.2488],
        [ 145.8659],
        [  28.1982],
        [  -7.5141],
        [ -52.2488],
        [-102.2488],
        [ 178.1982],
        [  42.4859],
        [  -2.2488],
        [ -52.2488],
        [-101.0458],
        [ 192.4859],
        [  47.7512],
        [  -2.2488],
        [ -51.0458],
        [-100.3584]])

In [645]:
test = np.zeros((25,5))
test[0:5], test[6:10], test[12:15], test[18:20], test[24:25] = \
.99*p, .99*p[1:5], .99*p[2:5], .99*p[3:5], .99*p[4:5]
A = test.copy()
for i in range(25):
    for j in range(5):
        if not A[i][0] == 0:
            A[i][0] = round(1-A[i][0],4)
            A[i][j] = round(A[i][j],4)

A[0:5,1:5] = -1*A[0:5,1:5]
A[5]=[-.856,.866,0,0,0]
A[6]=[-.5881,.732,-.134,0,0]
A[7]=[-.3201,.732,-.268,-.134,0]
A[8]=[-.1414,.8214,-.268,-.268,-.134]
A[9]=[-.1414,.8214,-.268,-.268,-.134]
A[10]=[-.5881,-.268,.866,0,0]
A[11]=[-.3201,-.268,.732,-.134,0]
A[12]=[-.1414,-.1786,.732,-.268,-.134]
A[13]=[-.1414,-.1786,.732,-.268,-.134]
A[14]=[-.1414,-.1786,.732,-.268,-.134]
A[15]=[-.3201,-.268,-.268,.866,0]
A[16]=[-.1414,-.1786,-.268,.732,-.134]
A[17]=[-.1414,-.1786,-.268,.732,-.134]
A[18]=[-.1414,-.1786,-.268,.732,-.134]
A[19]=[-.1414,-.1786,-.268,.732,-.134]
A[20]=[-.1414,-.1786,-.268,-.268,.866]
A[21]=[-.1414,-.1786,-.268,-.268,.866]
A[22]=[-.1414,-.1786,-.268,-.268,.866]
A[23]=[-.1414,-.1786,-.268,-.268,.866]
A[24]=[-.1414,-.1786,-.268,-.268,.866]
A

array([[ 0.01  , -0.    , -0.    , -0.    , -0.    ],
       [ 0.144 , -0.134 , -0.    , -0.    , -0.    ],
       [ 0.4119, -0.268 , -0.134 , -0.    , -0.    ],
       [ 0.6799, -0.268 , -0.268 , -0.134 , -0.    ],
       [ 0.8586, -0.1786, -0.268 , -0.268 , -0.134 ],
       [-0.856 ,  0.866 ,  0.    ,  0.    ,  0.    ],
       [-0.5881,  0.732 , -0.134 ,  0.    ,  0.    ],
       [-0.3201,  0.732 , -0.268 , -0.134 ,  0.    ],
       [-0.1414,  0.8214, -0.268 , -0.268 , -0.134 ],
       [-0.1414,  0.8214, -0.268 , -0.268 , -0.134 ],
       [-0.5881, -0.268 ,  0.866 ,  0.    ,  0.    ],
       [-0.3201, -0.268 ,  0.732 , -0.134 ,  0.    ],
       [-0.1414, -0.1786,  0.732 , -0.268 , -0.134 ],
       [-0.1414, -0.1786,  0.732 , -0.268 , -0.134 ],
       [-0.1414, -0.1786,  0.732 , -0.268 , -0.134 ],
       [-0.3201, -0.268 , -0.268 ,  0.866 ,  0.    ],
       [-0.1414, -0.1786, -0.268 ,  0.732 , -0.134 ],
       [-0.1414, -0.1786, -0.268 ,  0.732 , -0.134 ],
       [-0.1414, -0.1786, -0

In [647]:
print('A dim:', A.shape, 'b dim:',b.T.shape, 'c dim:',c.shape)

A dim: (25, 5) b dim: (25, 1) c dim: (5,)


In [656]:
res = sp.optimize.linprog(c, A_ub=-A, b_ub=-b.T, method='interior-point')

In [657]:
res

     con: array([], dtype=float64)
     fun: 20365.011329067012
 message: 'Optimization terminated successfully.'
     nit: 8
   slack: array([  3.92980710e+01,   9.55329919e+01,   5.96328316e+01,
         2.88583499e+01,   2.46692622e-08,  -1.15810508e-08,
         6.40998396e+01,   3.33253580e+01,   4.46700809e+00,
         4.92017081e+01,  -3.20517302e-08,   6.92255183e+01,
         4.03671684e+01,   8.51018684e+01,   1.35101868e+02,
        -3.87887837e-08,   7.11416501e+01,   1.15876350e+02,
         1.65876350e+02,   2.14673350e+02,  -4.02672811e-08,
         1.44734700e+02,   1.94734700e+02,   2.43531700e+02,
         2.92844300e+02])
  status: 0
 success: True
       x: array([ 3929.80710052,  3984.27410858,  4070.17426892,  4150.94875059,
        4229.80710046])

In [658]:
res.x

array([ 3929.80710052,  3984.27410858,  4070.17426892,  4150.94875059,
        4229.80710046])

(a)

$V(x) = \max_T E_x[\sum_{j=0}^{T-1} d(X_j) + r(X_T)]$

or in in english, max(reward when stop (T=0), don't stop(T>0))

$= max(r(X_0), E_x[d(x) + \sum_{j=0}^{T-1} d(X_j) + r(X_T)])$

$= max(r(X_0), d(x) + \sum_{y \in S}p(x,y)v(y))$

which is basically an FTA

(b)

$min \sum_x V(x)$

$s.t.$

$V(x) \geq max(r(X_0), d(x) + \sum_{y \in S}p(x,y)v(y))\ \forall x$

(a)

This is the memoryless property at work

$Exp\ R_i \sim Exp(\lambda_i)$

(b)

We want the minimum amount of time that we'll have to wait to see a doctor

$W = min_{1 \leq i \leq k} R_i \sim Exp(\sum_{i=0}^k \lambda_i)$

(c)

the crux of the solution to this problem relies on the fact that the doctors are independent

$P(R_1 < R_2) = \int_{0}^{\infty} P(t < R_2) dt$

$= \int_{0}^{\infty} P(R_1 < R_2 | R_1 = t) \lambda_1 exp(-\lambda, t) dt$

$= \int_{0}^{\infty} P(t < R_2 | R_1 = t) \lambda_1 exp(-\lambda, t) dt$

$= \int_{0}^{\infty} P(t < R_2) \lambda_1 exp(-\lambda, t) dt$

$= \frac{\lambda_i}{\sum_{j=1}^k \lambda_j}$

(d)

we rely on independence here too

$min_{j \neq i} R_j \sim Exp(\sum_{j=1}^k \lambda_j)$ is independent of $R_i \sim Exp(\lambda_i)$

so if we want to solve for $P(R_i < min_{j \neq i} R_j)$ we can use the same solution as in c

$= \frac{\lambda_i}{\sum_{j=1}^k \lambda_j}$

(e)

we can compute the expected amount of time spent as the expected time spent conditioned on $R_i < min_{j \neq i} R_j $

$E[time] =  \sum_{i=1}^k E[time|R_i < min_{j \neq i} R_j] P(R_i < min_{j \neq i} R_j )$

$=\sum_{i=1}^k (E[W | R_i < min_{j \neq i} R_j] + E[T_i | R_i < min_{j \neq i} R_j]) P(R_i < min_{j \neq i} R_j )$

$=\sum_{i=1}^k (E[W | R_i < min_{j \neq i} R_j] + E[T_i]) P(R_i < min_{j \neq i} R_j )$

$= \sum_{i=1}^k (E[W | R_i < min_{j \neq i} R_j] + \frac{1}{\lambda}) P(R_i < min_{j \neq i} R_j )$

$=(\frac{E[R_i I(R_i < min_{j \neq i} R_j)]}{P(R_i < min_{j \neq i} R_j)} + \frac{1}{\lambda} )P(R_i < min_{j \neq i} R_j )$

where we only want to integrate the region where $I(R_i < min_{j \neq i} R_j)$

and the probabilities in the demoninator of the left portion of the equation cancel the probability in the right

$= \int_{0}^{\infty}(\int_{0}^{x} y \lambda_j exp(\lambda_i y) dy) \lambda_j exp(-\sum_{j\neq i} \lambda_j x)dx * \frac{1}{\lambda_i}$

with the inner integral being the PDF of $R_i$

and $ \int_{0}^{\infty}\lambda_j exp(-\sum_{j\neq i} \lambda_j x)dx = \frac{\lambda_i}{\lambda_i + \lambda_j}$


$= Exp(\lambda_i) \frac{\lambda_i}{\lambda_i + \lambda_j}\frac{1}{\lambda_i}$

(a)

$Y_1 \sim exp(\lambda)$

$Y_1 \sim exp(\mu)$

such that $P(Y_1 > y, Y_2 > y) = exp(-y \lambda)exp(-y \mu) = exp(-y(\lambda + \mu))$

$min(Y_1, Y_2) \sim exp(\lambda + \mu) = E[(Y_1,Y_2)] = \frac{1}{\lambda+\mu}$


$\begin{equation}
  P_{ij}=\begin{cases}
    \frac{\mu}{\lambda+\mu}, & \text{if $j = i - 1$}.\\
    \frac{\lambda}{\lambda+\mu}, & \text{if $j = i + 1$}.
  \end{cases}
\end{equation}$

(b)

for $i \geq 0$

$E_i[T_j] = \frac{1}{\lambda+\mu} + E_{i-1}[T_j \frac{\mu}{\lambda+\mu}] + E_{i+1}[T_j \frac{\lambda}{\lambda+\mu}]$

(c)

$E_0[T_1] = \frac{1}{\lambda}$

whereas $E_{12} = E_1[T_2]$ or $E_{ij} = E_i[T_j]$

so $(\lambda + \mu)E_{12} = 1 + (E_{01} + E_{12}$

and

$\lambda E_{12} = 1 + \frac{\mu}{\lambda}$

$= E_{12} = \frac{1}{\lambda}[1+\frac{\mu}{\lambda}^1]$

we can prove this further by induction

we know that $E_{i, i+1} = \frac{1}{\lambda}[\sum_{k=0}^i \frac{\mu}{\lambda}^k]$ for i = 0, i = 1

$E_{j, j+1} = \frac{1}{\lambda + \mu} + \frac{\mu}{\lambda + \mu} E_{j-1, j+1}$

$(\lambda + \mu)E_{j, j+1} = 1 + \mu(E_{j-1, j}+E_{j, j+1})$

$\lambda E_{j, j+1} = 1 + \mu E_{j-1, j}$

$E_{j, j+1} = \frac{1}{\lambda} + \frac{\mu}{\lambda} E_{j-1, j}$

$ = \frac{1}{\lambda} + \frac{\mu}{\lambda}\frac{1}{\lambda}\sum_{k=0}^{j-1}\frac{\mu}{\lambda}^k$

$ = \frac{1}{\lambda} + [{\lambda}\sum_{k=0}^{j-1}\frac{\mu}{\lambda}^k]$

which concludes the proof

(d)

we can solve for the expected amount of time by solving into systems of linear equations given the expectations that we found earlier

$S_j = E_{0, j}$

such that $S_j = E_{0,1} + E_{0,1} + ... E_{j-1,j}$

= $1/2 + 1/2(3/2)$

= $1/1 + 1/2(3/2) + 1/2(3/2)^2$

so $S_{j+1} = (3/2)S_j + 1/2(j+1)$

$2S_{j+1} = 3S_j + j + 1$

thus $2X = 3; X = 3/2$

$S_j = a(3/2)^j + bj + c$

$S_0 = 0; 0 = a + c$

$S_1 = 1/2; 1/2 = 3/2a + b + c$

$S_2 = 7/4; 7/4 = 9/4 + 2b + c$

solving yields a = 3, b = -1, c = -3

$S_j = 3(3/2)^j - n - 3$